In [4]:

from base64 import b64encode
from bs4 import BeautifulSoup
from random import randint

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import _find_element
from selenium.webdriver.support.ui import WebDriverWait

#import pandas as pd
import pickle
import random
import re
import requests
import time
import urllib.parse

proxies_array = [
    {'host': '64.120.85.192', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Chicago Ilinois'},
    {'host': '8.29.125.84', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Columbus Ohio'},
    {'host': '173.234.204.209', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Dallas Texas'},
    {'host': '8.29.124.215', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Denver Colorado'},
    {'host': '167.160.106.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Houston Texas'},
    {'host': '192.171.229.113', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Irvine California'},
    {'host': '45.59.21.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Las Vegas Nevada'},
    {'host': '142.91.235.188', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Los Angeles California'},
    {'host': '108.62.137.20', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'New York City New York'},
    {'host': '216.107.136.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Oklahoma City Oklahoma'},
    {'host': '23.19.213.86', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Phoenix Arizona'},
]

def get_firefox_credentials(proxy):
    credentials = '{usr}:{pwd}'.format(**proxy)
    credentials = b64encode(credentials.encode('ascii')).decode('utf-8')
    
    return(credentials)

def get_firefox_profile(credentials):
    fp = webdriver.FirefoxProfile()
    fp.set_preference('network.proxy.type', 1)
    fp.set_preference('network.proxy.http', proxy['host'])
    fp.set_preference('network.proxy.http_port', int(proxy['port']))
    fp.set_preference('network.proxy.no_proxies_on', 'localhost, 127.0.0.1')
    fp.add_extension('closeproxy.xpi')

    # Set up the credentials
    fp.set_preference('extensions.closeproxyauth.authtoken', credentials)

    #set some privacy settings
    fp.set_preference( "places.history.enabled", False )
    fp.set_preference( "privacy.clearOnShutdown.offlineApps", True )
    fp.set_preference( "privacy.clearOnShutdown.passwords", True )
    fp.set_preference( "privacy.clearOnShutdown.siteSettings", True )
    fp.set_preference( "privacy.sanitize.sanitizeOnShutdown", True )
    fp.set_preference( "signon.rememberSignons", False )
    fp.set_preference( "network.cookie.lifetimePolicy", 2 )
    fp.set_preference( "network.dns.disablePrefetch", True )
    fp.set_preference( "network.http.sendRefererHeader", 0 )

    #if you're really hardcore about your security
    #js can be used to reveal your true i.p.
    #fp.set_preference( "javascript.enabled", False )

    #get a huge speed increase by not downloading images
    #fp.set_preference( "permissions.default.image", 2 )
    
    return(fp)

def get_firefox_page(proxy, url):
    credentials = get_firefox_credentials(proxy)
    fp = get_firefox_profile(credentials)
    driver = webdriver.Firefox(fp)

    # Set the timeout information
    driver.set_page_load_timeout(50)

    finished = 0
    fails = 0
    while finished == 0 and fails < 8:
        try:
            driver.get(url)
            finished = 1
        except Exception as e:
            print(str(e))
            fails = fails + 1
            time.sleep(5)
    
    return(driver)

class text_to_change(object):
    def __init__(self, locator, text):
        self.locator = locator
        self.text = text

    def __call__(self, driver):
        actual_text = _find_element(driver, self.locator).text
        return actual_text != self.text

http://www.rvt.com/New-and-Used-RVs-For-Sale-On-RVT.com/results?private_only=1


In [5]:
siteUrl = "http://www.rvt.com/rvsearchform.php"
print(siteUrl)

proxy = proxies_array[randint(0, len(proxies_array)-1)]

driver = get_firefox_page(proxy, siteUrl)

http://www.rvt.com/rvsearchform.php
Message: Error loading page, timed out (checkLoad)

Message: Error loading page, timed out (checkLoad)

Message: Error loading page, timed out (checkLoad)



In [6]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<input id="private" class="std-formcheck clear" name="private_only" type="checkbox" value="1">
    checkboxCss = '#private'
    
    # Wait for search to show up
    checkboxTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, checkboxCss))
    )
    
    print(driver.find_element_by_css_selector(checkboxCss).get_attribute('outerHTML'))
    checkboxTag.click()
except Exception as e:
    print(str(e))

<input id="private" class="std-formcheck clear" name="private_only" value="1" type="checkbox">


In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<button type="submit" class="btn" name="submit" value="Search Now">Search Now</button>
    buttonCss = '#searchform > button.btn'
    
    # Wait for search to show up
    buttonTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, buttonCss))
    )
    
    print(driver.find_element_by_css_selector(buttonCss).get_attribute('outerHTML'))
    buttonTag.click()
except Exception as e:
    print(str(e))

<button type="submit" class="btn" name="submit" value="Search Now">Search Now</button>


In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<a class="mdwd cboxElement" href="http://www.rvt.com/search-top-type.php?&amp;private_only=1&amp;show=all">+ Show All</a>
    showCss = 'li.filters > ul > li > a.mdwd.cboxElement'
    
    # Wait for "+ Show All" to show up
    showTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, showCss))
    )
    
    print(driver.find_element_by_css_selector(showCss).get_attribute('outerHTML'))
    showTag.click()
except Exception as e:
    print(str(e))

<a class="mdwd cboxElement" href="http://www.rvt.com/search-top-type.php?&amp;private_only=1&amp;show=all">+ Show All</a>


In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<ul>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-5th-Wheel-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=5th Wheels');">5th Wheels <span>(593)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Bus-Conversion-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Bus Conversions');">Bus Conversions <span>(26)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Cargo-Trailer-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Cargo Trailers');">Cargo Trailers <span>(1)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Class-A---Diesel-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Class A - Diesel');">Class A - Diesel <span>(738)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Class-A---Gas-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Class A - Gas');">Class A - Gas <span>(481)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Class-B-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Class B');">Class B <span>(71)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Class-B-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Class B');">Class B <span>(27)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Class-C-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Class C');">Class C <span>(274)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Conversion-Van-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Conversion Van');">Conversion Van <span>(3)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Folding-Trailer-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Folding Trailers');">Folding Trailers <span>(18)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Ice-Fish-House-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Ice Fish House');">Ice Fish House <span>(1)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Park-Model-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Park Models');">Park Models <span>(20)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-SUV-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=SUV');">Suv <span>(13)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Toterhome-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Toterhomes');">Toterhomes <span>(9)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Tow-Behind-Cars-For-Sale-On-RVT.com/results?private_only=1&amp;type=Tow Behind Cars');">Tow Behind Cars <span>(29)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Toy-Hauler-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Toy Haulers');">Toy Haulers <span>(28)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Toy-Haulers-5th-Wheel-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Toy Haulers 5th Wheels');">Toy Haulers 5th Wheels <span>(92)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Toy-Haulers-Travel-Trailer-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Toy Haulers Travel Trailers');">Toy Haulers Travel Trailers <span>(33)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Travel-Trailer-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Travel Trailers');">Travel Trailers <span>(464)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Travel-Trailers---Hybrid-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Travel Trailers - Hybrid');">Travel Trailers - Hybrid <span>(14)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Truck-Camper-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Truck Campers');">Truck Campers <span>(32)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Trucks-For-Sale-On-RVT.com/results?private_only=1&amp;type=Trucks');">Trucks <span>(42)</span></a></li>
  <li><a href="JavaScript:void(0);" onclick="openInParent('http://www.rvt.com/New-and-Used-Utility-Trailer-RVs-For-Sale-On-RVT.com/results?private_only=1&amp;type=Utility Trailers');">Utility Trailers <span>(2)</span></a></li>
</ul>
    showCss = 'li.filters > ul > li > a.mdwd.cboxElement'
    
    # Wait for "+ Show All" to show up
    showTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, showCss))
    )
    
    print(driver.find_element_by_css_selector(showCss).get_attribute('outerHTML'))
    showTag.click()
except Exception as e:
    print(str(e))

<a class="mdwd cboxElement" href="http://www.rvt.com/search-top-type.php?&amp;private_only=1&amp;show=all">+ Show All</a>


In [ ]:
# Retrieve the page with tag results and set it up to be scraped
proxy_url = 'http://{usr}:{pwd}@{host}:{port}/'.format(**proxy)
#sitePage = requests.get(url=siteUrl, proxies={'http': proxy_url})
#siteSoup = BeautifulSoup(sitePage.content, 'lxml')

links = siteSoup.select('div.result-col > div.post-search > ul > li > a')
#print(sitePage.content)